In [4]:
# import sample data: Loan screening data for classification 
import pandas as pd

df = pd.read_csv('./data/av_loan_u6lujuX_CVtuZ9i.csv', header=0)
X  = df.iloc[:, :-1]            # 最終列が審査結果のため最終列以前を特徴量Xとして読込
ID = X.iloc[:, [0]]             # 最初列がPK（Loan_ID）なのでID情報としてセット
X  = X.drop('Loan_ID', axis=1)  # 1列目(Loan_ID)は特徴量ベクトルから削除(drop関数におけるaxis=1は列方向)
y  = df.iloc[:, [-1]]           # 最終列を正解データとして読込

# check the shape
print('----------------------------------------------------------------------------------------')
print('Raw shape: (%i,%i)' %df.shape)
print('ID shape: (%i,%i)' %ID.shape)
print('X shape: (%i,%i)' %X.shape)
print('y shape: (%i,%i)' %y.shape)
print('----------------------------------------------------------------------------------------')
print('Check the null count of the target variable: %i' % y.isnull().sum())
print('----------------------------------------------------------------------------------------')

#converting string to number(binary flag)
#ローン審査でNOとなったサンプルを1(正例)として変換
class_mapping = {'N':1, 'Y':0}
y_new = y.copy()
y_new.loc[:,'Loan_Status'] = y_new['Loan_Status'].map(class_mapping)
print(y_new.groupby(['Loan_Status']).size())
print('----------------------------------------------------------------------------------------')
print(ID.join(X).join(y_new).dtypes)
ID.join(X).join(y_new).head()

----------------------------------------------------------------------------------------
Raw shape: (614,13)
ID shape: (614,1)
X shape: (614,11)
y shape: (614,1)
----------------------------------------------------------------------------------------
Check the null count of the target variable: 0
----------------------------------------------------------------------------------------
Loan_Status
0    422
1    192
dtype: int64
----------------------------------------------------------------------------------------
Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status            int64
dtype: object


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,0
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,1
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,0
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,0
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,0


In [6]:
ohe_columns = ['Dependents','Gender','Married','Education','Self_Employed','Property_Area']
X_ohe = pd.get_dummies(X, dummy_na=True, columns=ohe_columns)
print('X_ohe shape:(%i,%i)'% X_ohe.shape)
X_ohe.head()

X_ohe shape:(614,26)


,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Dependents_nan,...,Education_Graduate,Education_Not Graduate,Education_nan,Self_Employed_No,Self_Employed_Yes,Self_Employed_nan,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Property_Area_nan
0,5849,0.0,NaN,360.0,1.0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0
1,4583,1508.0,128.0,360.0,1.0,0,1,0,0,0,...,1,0,0,1,0,0,1,0,0,0
2,3000,0.0,66.0,360.0,1.0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
3,2583,2358.0,120.0,360.0,1.0,1,0,0,0,0,...,0,1,0,1,0,0,0,0,1,0
4,6000,0.0,141.0,360.0,1.0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0


In [8]:
from sklearn.preprocessing import Imputer

#欠損値NaNを平均値 (mean)で置換
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(X_ohe)

#学習済みImputerを適用しX_newの欠損値を置換
X_ohe_columns = X_ohe.columns.values
X_ohe = pd.DataFrame(imp.transform(X_ohe), columns=X_ohe_columns)

#結果表示
X_ohe.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Dependents_nan,...,Education_Graduate,Education_Not Graduate,Education_nan,Self_Employed_No,Self_Employed_Yes,Self_Employed_nan,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Property_Area_nan
0,5849.0,0.0,146.412162,360.0,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,4583.0,1508.0,128.000000,360.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,3000.0,0.0,66.000000,360.0,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,2583.0,2358.0,120.000000,360.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,6000.0,0.0,141.000000,360.0,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import GradientBoostingClassifier

selector = RFE(GradientBoostingClassifier(random_state=1), n_features_to_select=10, step=.05)
selector.fit(X_ohe, y.as_matrix().ravel())

X_fin = pd.DataFrame(selector.transform(X_ohe), columns=X_ohe_columns[selector.support_])
print('X_fin shape:(%i,%i)' % X_fin.shape)
X_fin.head()

X_fin shape:(614,10)


,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Dependents_1,Married_No,Married_nan,Self_Employed_nan,Property_Area_Semiurban
0,5849.0,0.0,146.412162,360.0,1.0,0.0,1.0,0.0,0.0,0.0
1,4583.0,1508.0,128.000000,360.0,1.0,1.0,0.0,0.0,0.0,0.0
2,3000.0,0.0,66.000000,360.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2583.0,2358.0,120.000000,360.0,1.0,0.0,0.0,0.0,0.0,0.0
4,6000.0,0.0,141.000000,360.0,1.0,0.0,1.0,0.0,0.0,0.0


In [12]:
import pandas as pd

#import sample data
#Loan screening data for classification
df_s = pd.read_csv('./data/av_loan_test_Y3wMUE5_7gLdaTN.csv', header=0)

ID_s = df_s.iloc[:,[0]]
X_s = df_s.drop('Loan_ID', axis = 1)

#check the shape
print('----------------------------------------------------------------------------------------')
print('Raw shape: (%i,%i)' %df_s.shape)
print('ID shape: (%i,%i)' %ID_s.shape)
print('X shape: (%i,%i)' %X_s.shape)
print(X_s.dtypes)

----------------------------------------------------------------------------------------
Raw shape: (333,12)
ID shape: (333,1)
X shape: (333,11)
Gender                object
Married               object
Dependents           float64
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome      int64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
dtype: object


In [13]:
ohe_columns = ['Dependents','Gender','Married','Education','Self_Employed','Property_Area']
X_ohe_s = pd.get_dummies(X_s, dummy_na = True, columns=ohe_columns)
print('X_ohe_s shape:(%i,%i)' % X_ohe_s.shape)
X_ohe_s.head()

X_ohe_s shape:(333,26)


,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Dependents_0.0,Dependents_1.0,Dependents_2.0,Dependents_nan,Gender_Female,...,Education_Graduate,Education_Not Graduate,Education_nan,Self_Employed_No,Self_Employed_Yes,Self_Employed_nan,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Property_Area_nan
0,5720,0,110.0,360.0,1.0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0
1,3076,1500,126.0,360.0,1.0,0,1,0,0,0,...,1,0,0,1,0,0,0,0,1,0
2,5000,1800,208.0,360.0,1.0,0,0,1,0,0,...,1,0,0,1,0,0,0,0,1,0
3,2340,2546,100.0,360.0,NaN,0,0,1,0,0,...,1,0,0,1,0,0,0,0,1,0
4,3276,0,78.0,360.0,1.0,1,0,0,0,0,...,0,1,0,1,0,0,0,0,1,0


In [16]:
cols_model = set(X_ohe.columns.values)
cols_score = set(X_ohe_s.columns.values)

#モデルにはあったスコアにはないデータ項目
diff1 = cols_model - cols_score
print('モデルのみに存在する項目:%s' % diff1)

#スコアにはあるがモデルにはなかったデータ項目
diff2 = cols_score - cols_model
print('スコアのみに存在する項目:%s' % diff2)

モデルのみに存在する項目:{'Dependents_2', 'Dependents_0', 'Dependents_1', 'Dependents_3+'}
スコアのみに存在する項目:{'Dependents_2.0', 'Dependents_0.0', 'Gender_Unknown', 'Dependents_1.0'}
